In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import keras
import pandas as pd
from keras.layers import GRU
import tensorflow as tf

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/datasets/carla/carla_data.csv')
data = data.iloc[:, :4]  # 데이터의 첫 4개 열만 사용

# 입력 데이터와 타겟 데이터로 나누기
input_data = []
target_data = []

# 각 id 그룹에 대해 반복
for group_id in data['id'].unique():
    group_data = data[data['id'] == group_id].reset_index(drop=False)
    num_samples = len(group_data)  # group_datat : 각 id에 대하는 좌표들의 갯수

    # 각 그룹 내에서 10개의 연속된 데이터를 입력으로, 그 다음 데이터의 좌표를 타겟으로 사용
    for i in range(10, num_samples):
        input_data.append(group_data.iloc[i-10:i][['X', 'Y','point_id']].to_numpy())
        target_data.append(group_data.iloc[i][['X', 'Y','point_id']].to_numpy())

# numpy 배열로 변환
input_data = np.array(input_data)
target_data = np.array(target_data)

# 학습용과 검증용으로 데이터를 나눔
train_input, test_input, train_target, test_target = train_test_split(input_data, target_data, test_size=0.2, random_state=42)


In [ ]:
def weighted_mse_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    loss = mse(y_true, y_pred)
    
    # 벽에 가까운 좌표에 대한 가중치 설정
    wall_threshold = 5  # 이 값은 벽에 가까운 좌표를 정의하는 데 사용되며, 사용자가 조정할 수 있습니다.
    weights = tf.where(
        tf.logical_or(
            tf.less(y_true[:, :2], wall_threshold),
            tf.greater(y_true[:, :2], 72 - wall_threshold)
        ),
        2.0,  # 벽에 가까운 좌표에 대한 가중치
        1.0   # 그 외의 좌표에 대한 가중치
    )
    
    # 가중치를 적용한 손실 값을 계산
    weighted_loss = tf.multiply(loss, weights)
    
    return tf.reduce_mean(weighted_loss)

In [ ]:
model = Sequential()
model.add(GRU(32, input_shape=(10, 3), return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(64, return_sequences=False))
model.add(Dense(3))

# 모델 컴파일

model.compile(loss=weighted_mse_loss, optimizer='adam')


# EarlyStopping 콜백 설정
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
history = model.fit(input_data, target_data, epochs=1000, batch_size=16, validation_split=0.2)

In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/model/carla_GRU_model.h5')